## Preprocessing

In [2]:
!pip install tensorflow

  Using cached tensorflow-2.14.0-cp310-cp310-macosx_10_15_x86_64.whl (229.6 MB)
  Using cached absl_py-2.0.0-py3-none-any.whl (130 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)
  Using cached gast-0.5.4-py3-none-any.whl (19 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached h5py-3.9.0-cp310-cp310-macosx_10_9_x86_64.whl (3.2 MB)
  Using cached libclang-16.0.6-py2.py3-none-macosx_10_9_x86_64.whl (24.5 MB)
  Using cached ml_dtypes-0.2.0-cp310-cp310-macosx_10_9_universal2.whl (1.2 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached protobuf-4.24.3-cp37-abi3-macosx_10_9_universal2.whl (409 kB)
  Using cached termcolor-2.3.0-py3-none-any.whl (6.9 kB)
  Using cached wrapt-1.14.1-cp310-cp310-macosx_10_9_x86_64.whl (35 kB)
  Using cached tensorflow_io_gcs_filesystem-0.34.0-cp310-cp310-macosx_10_14_x86_64.whl (1.7 MB)
  Using cached grpcio-1.58.0.tar.gz (24.8 MB)


In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np
from scipy.stats import pointbiserialr, chi2_contingency, ttest_ind
import joblib

In [3]:
#  Import and read the QChat data.
QChat_df = pd.read_csv("Resources/Toddler Autism dataset July 2018.csv")
QChat_df.head()

,Case_No,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,Age_Mons,Qchat-10-Score,Sex,Ethnicity,Jaundice,Family_mem_with_ASD,Who completed the test,Class/ASD Traits
0,1,0,0,0,0,0,0,1,1,0,1,28,3,f,middle eastern,yes,no,family member,No
1,2,1,1,0,0,0,1,1,0,0,0,36,4,m,White European,yes,no,family member,Yes
2,3,1,0,0,0,0,0,1,1,0,1,36,4,m,middle eastern,yes,no,family member,Yes
3,4,1,1,1,1,1,1,1,1,1,1,24,10,m,Hispanic,no,no,family member,Yes
4,5,1,1,0,1,1,1,1,1,1,1,20,9,f,White European,no,yes,family member,Yes


In [4]:
QChat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1054 entries, 0 to 1053
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Case_No                 1054 non-null   int64 
 1   A1                      1054 non-null   int64 
 2   A2                      1054 non-null   int64 
 3   A3                      1054 non-null   int64 
 4   A4                      1054 non-null   int64 
 5   A5                      1054 non-null   int64 
 6   A6                      1054 non-null   int64 
 7   A7                      1054 non-null   int64 
 8   A8                      1054 non-null   int64 
 9   A9                      1054 non-null   int64 
 10  A10                     1054 non-null   int64 
 11  Age_Mons                1054 non-null   int64 
 12  Qchat-10-Score          1054 non-null   int64 
 13  Sex                     1054 non-null   object
 14  Ethnicity               1054 non-null   object
 15  Jaun

In [5]:
QChat_df['Class/ASD Traits '].value_counts()

Yes    728
No     326
Name: Class/ASD Traits , dtype: int64

In [6]:
# Update Yes/No in Class/ASD Traits column to 1/0.
# Change 0 to 1 and 1 to 0 in A10 - this is a reverse question
QChat_df['Class/ASD Traits '] = QChat_df['Class/ASD Traits '].map({'Yes': 1, 'No': 0})
QChat_df['A10'] = 1 - QChat_df['A10']
QChat_df

,Case_No,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,Age_Mons,Qchat-10-Score,Sex,Ethnicity,Jaundice,Family_mem_with_ASD,Who completed the test,Class/ASD Traits
0,1,0,0,0,0,0,0,1,1,0,0,28,3,f,middle eastern,yes,no,family member,0
1,2,1,1,0,0,0,1,1,0,0,1,36,4,m,White European,yes,no,family member,1
2,3,1,0,0,0,0,0,1,1,0,0,36,4,m,middle eastern,yes,no,family member,1
3,4,1,1,1,1,1,1,1,1,1,0,24,10,m,Hispanic,no,no,family member,1
4,5,1,1,0,1,1,1,1,1,1,0,20,9,f,White European,no,yes,family member,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1049,1050,0,0,0,0,0,0,0,0,0,0,24,1,f,White European,no,yes,family member,0
1050,1051,0,0,1,1,1,0,1,0,1,1,12,5,m,black,yes,no,family member,1
1051,1052,1,0,1,1,1,1,1,1,1,0,18,9,m,middle eastern,yes,no,family member,1
1052,1053,1,0,0,0,0,0,0,1,0,0,19,3,m,White European,no,yes,family member,0


In [7]:
QChat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1054 entries, 0 to 1053
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Case_No                 1054 non-null   int64 
 1   A1                      1054 non-null   int64 
 2   A2                      1054 non-null   int64 
 3   A3                      1054 non-null   int64 
 4   A4                      1054 non-null   int64 
 5   A5                      1054 non-null   int64 
 6   A6                      1054 non-null   int64 
 7   A7                      1054 non-null   int64 
 8   A8                      1054 non-null   int64 
 9   A9                      1054 non-null   int64 
 10  A10                     1054 non-null   int64 
 11  Age_Mons                1054 non-null   int64 
 12  Qchat-10-Score          1054 non-null   int64 
 13  Sex                     1054 non-null   object
 14  Ethnicity               1054 non-null   object
 15  Jaun

In [8]:
# Convert categorical data to numeric with `pd.get_dummies`
category_dummies = pd.get_dummies(QChat_df)

In [9]:
category_dummies
# category_dummies.info()
# print(category_dummies.isnull().sum())
# print(np.isinf(category_dummies).sum())

,Case_No,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,Ethnicity_south asian,Jaundice_no,Jaundice_yes,Family_mem_with_ASD_no,Family_mem_with_ASD_yes,Who completed the test_Health Care Professional,Who completed the test_Health care professional,Who completed the test_Others,Who completed the test_Self,Who completed the test_family member
0,1,0,0,0,0,0,0,1,1,0,...,0,0,1,1,0,0,0,0,0,1
1,2,1,1,0,0,0,1,1,0,0,...,0,0,1,1,0,0,0,0,0,1
2,3,1,0,0,0,0,0,1,1,0,...,0,0,1,1,0,0,0,0,0,1
3,4,1,1,1,1,1,1,1,1,1,...,0,1,0,1,0,0,0,0,0,1
4,5,1,1,0,1,1,1,1,1,1,...,0,1,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1049,1050,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,1
1050,1051,0,0,1,1,1,0,1,0,1,...,0,0,1,1,0,0,0,0,0,1
1051,1052,1,0,1,1,1,1,1,1,1,...,0,0,1,1,0,0,0,0,0,1
1052,1053,1,0,0,0,0,0,0,1,0,...,0,1,0,0,1,0,0,0,0,1


# Correlations?

In [10]:
# Use Phi coefficient to find relationship between QChat data and ASD Traits. 
correlation_df = category_dummies[['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'Class/ASD Traits ']]

numbers = range(1,11)
survey_correlations_list = []

for number in numbers:
    question = f'A{number}'
    phi_coefficient, p_value = pointbiserialr(correlation_df[question], correlation_df['Class/ASD Traits '])
    question_data = {'question': question, 'Phi Coefficient (φ)': phi_coefficient, 'P-value': p_value}
    survey_correlations_list.append(question_data)

survey_correlations = pd.DataFrame(survey_correlations_list)
survey_correlations


,question,Phi Coefficient (φ),P-value
0,A1,0.503810,6.336049e-69
1,A2,0.463467,3.036443e-57
2,A3,0.409701,6.325776e-44
3,A4,0.505204,2.342106e-69
4,A5,0.563297,2.772662e-89
5,A6,0.569424,1.270830e-91
6,A7,0.563177,3.078939e-89
7,A8,0.427155,5.467014e-48
8,A9,0.577336,1.024092e-94
9,A10,-0.179833,4.116972e-09


In [11]:
# Use Phi coefficient to find relationship between binary demographic data and ASD Traits. 
correlation2_df = category_dummies[[
    'Jaundice_yes', 
    'Family_mem_with_ASD_yes', 
    'Sex_m', 
    'Class/ASD Traits ']]

columns = ['Jaundice_yes', 'Family_mem_with_ASD_yes', 'Sex_m']
survey_correlations_list2 = []

for column in columns:
    phi_coefficient, p_value = pointbiserialr(correlation2_df[column], correlation2_df['Class/ASD Traits '])
    question_data = {'question': column, 'Phi Coefficient (φ)': phi_coefficient, 'P-value': p_value}
    survey_correlations_list2.append(question_data)

demographic_correlations_df = pd.DataFrame(survey_correlations_list2)
demographic_correlations_df

,question,Phi Coefficient (φ),P-value
0,Jaundice_yes,0.074060,0.016180
1,Family_mem_with_ASD_yes,-0.013503,0.661485
2,Sex_m,0.117664,0.000129


In [12]:
# Use independent t-test for age and total score.
correlation3_df = category_dummies[[
    'Qchat-10-Score', 
    'Age_Mons', 
    'Class/ASD Traits ']]

# Split the data into two groups based on the binary column (0 and 1)
demo_columns = [correlation3_df['Age_Mons'], correlation3_df['Qchat-10-Score']]
binary_column = correlation3_df['Class/ASD Traits ']
results_list = []

for i, column in enumerate(demo_columns):
    group_0 = column[binary_column == 0]
    group_1 = column[binary_column == 1]

    # Perform an independent samples t-test
    t_statistic, p_value = ttest_ind(group_0, group_1)
    result_dict = {
        'question': correlation3_df.columns[i],
        'T-Statistic': t_statistic,
        'P-value': p_value}
    results_list.append(result_dict)

demographic_correlations2_df = pd.DataFrame(results_list)
demographic_correlations2_df

,question,T-Statistic,P-value
0,Qchat-10-Score,-2.172494,3.004116e-02
1,Age_Mons,-44.867971,1.547748e-246


# Prep all data to train

In [13]:
# Split our preprocessed data into our features and target arrays
y = QChat_df['Class/ASD Traits ']
X = category_dummies.drop(['Class/ASD Traits '], axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [14]:
X

,Case_No,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,Ethnicity_south asian,Jaundice_no,Jaundice_yes,Family_mem_with_ASD_no,Family_mem_with_ASD_yes,Who completed the test_Health Care Professional,Who completed the test_Health care professional,Who completed the test_Others,Who completed the test_Self,Who completed the test_family member
0,1,0,0,0,0,0,0,1,1,0,...,0,0,1,1,0,0,0,0,0,1
1,2,1,1,0,0,0,1,1,0,0,...,0,0,1,1,0,0,0,0,0,1
2,3,1,0,0,0,0,0,1,1,0,...,0,0,1,1,0,0,0,0,0,1
3,4,1,1,1,1,1,1,1,1,1,...,0,1,0,1,0,0,0,0,0,1
4,5,1,1,0,1,1,1,1,1,1,...,0,1,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1049,1050,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,1
1050,1051,0,0,1,1,1,0,1,0,1,...,0,0,1,1,0,0,0,0,0,1
1051,1052,1,0,1,1,1,1,1,1,1,...,0,0,1,1,0,0,0,0,0,1
1052,1053,1,0,0,0,0,0,0,1,0,...,0,1,0,0,1,0,0,0,0,1


In [15]:
y

0       0
1       1
2       1
3       1
4       1
       ..
1049    0
1050    1
1051    1
1052    0
1053    1
Name: Class/ASD Traits , Length: 1054, dtype: int64

In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 8

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 288       
                                                                 
 dense_1 (Dense)             (None, 8)                 72        
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 369 (1.44 KB)
Trainable params: 369 (1.44 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
25/25 [==============================] - 2s 3ms/step - loss: 0.5986 - accuracy: 0.6848
Epoch 2/100
25/25 [==============================] - 0s 5ms/step - loss: 0.5514 - accuracy: 0.6899
Epoch 3/100
25/25 [==============================] - 0s 4ms/step - loss: 0.5083 - accuracy: 0.6911
Epoch 4/100
25/25 [==============================] - 0s 3ms/step - loss: 0.4691 - accuracy: 0.6911
Epoch 5/100
25/25 [==============================] - 0s 5ms/step - loss: 0.4341 - accuracy: 0.6911
Epoch 6/100
25/25 [==============================] - 0s 3ms/step - loss: 0.4031 - accuracy: 0.6975
Epoch 7/100
25/25 [==============================] - 0s 7ms/step - loss: 0.3762 - accuracy: 0.7772
Epoch 8/100
25/25 [==============================] - 0s 6ms/step - loss: 0.3527 - accuracy: 0.8658
Epoch 9/100
25/25 [==============================] - 0s 6ms/step - loss: 0.3316 - accuracy: 0.8899
Epoch 10/100
25/25 [==============================] - 0s 4ms/step - loss: 0.3116 - accuracy: 0.9076
Epoch 11/

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

9/9 - 0s - loss: 0.0150 - accuracy: 0.9924 - 354ms/epoch - 39ms/step
Loss: 0.01498386450111866, Accuracy: 0.9924242496490479


In [21]:
# Export our model to HDF5 file
nn.save('Resources/QChatNN.h5')

c:\Users\Jessi\anaconda3\envs\dev\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Optimize the Model

In [22]:
QChat_df = pd.read_csv("Resources/Toddler Autism dataset July 2018.csv")
QChat_df['Class/ASD Traits '] = QChat_df['Class/ASD Traits '].map({'Yes': 1, 'No': 0})
# Drop the non-beneficial columns,  'Case_No', 'Qchat-10-Score', 'Sex', 'Ethnicity', 'Who completed the test'
# Remaining columns: Age_Mons', 'Jaundice', 'Family_mem_with_ASD'
QChat_df = QChat_df.drop(columns=['Case_No', 'Who completed the test', 'Ethnicity', 'Qchat-10-Score', 'Sex', 'Jaundice', 'Family_mem_with_ASD'])
QChat_df.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,Age_Mons,Class/ASD Traits
0,0,0,0,0,0,0,1,1,0,1,28,0
1,1,1,0,0,0,1,1,0,0,0,36,1
2,1,0,0,0,0,0,1,1,0,1,36,1
3,1,1,1,1,1,1,1,1,1,1,24,1
4,1,1,0,1,1,1,1,1,1,1,20,1


In [23]:
# Determine the number of unique values in each column.
QChat_df.nunique()

A1                    2
A2                    2
A3                    2
A4                    2
A5                    2
A6                    2
A7                    2
A8                    2
A9                    2
A10                   2
Age_Mons             25
Class/ASD Traits      2
dtype: int64

In [24]:
# Convert categorical data to numeric with `pd.get_dummies`
category_dummies = pd.get_dummies(QChat_df)
category_dummies

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,Age_Mons,Class/ASD Traits
0,0,0,0,0,0,0,1,1,0,1,28,0
1,1,1,0,0,0,1,1,0,0,0,36,1
2,1,0,0,0,0,0,1,1,0,1,36,1
3,1,1,1,1,1,1,1,1,1,1,24,1
4,1,1,0,1,1,1,1,1,1,1,20,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1049,0,0,0,0,0,0,0,0,0,1,24,0
1050,0,0,1,1,1,0,1,0,1,0,12,1
1051,1,0,1,1,1,1,1,1,1,1,18,1
1052,1,0,0,0,0,0,0,1,0,1,19,0


In [25]:
# Split our preprocessed data into our features and target arrays
y = QChat_df['Class/ASD Traits ']
X = category_dummies.drop(['Class/ASD Traits '], axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Save the scaler to a file
joblib.dump(scaler, 'Resources/scaler.joblib')

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [26]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 8

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 8)                 96        
                                                                 
 dense_4 (Dense)             (None, 8)                 72        
                                                                 
 dense_5 (Dense)             (None, 1)                 9         
                                                                 
Total params: 177 (708.00 Byte)
Trainable params: 177 (708.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
25/25 [==============================] - 1s 2ms/step - loss: 0.8316 - accuracy: 0.3582
Epoch 2/50
25/25 [==============================] - 0s 3ms/step - loss: 0.7107 - accuracy: 0.4937
Epoch 3/50
25/25 [==============================] - 0s 3ms/step - loss: 0.6170 - accuracy: 0.6608
Epoch 4/50
25/25 [==============================] - 0s 3ms/step - loss: 0.5362 - accuracy: 0.7823
Epoch 5/50
25/25 [==============================] - 0s 3ms/step - loss: 0.4650 - accuracy: 0.8443
Epoch 6/50
25/25 [==============================] - 0s 3ms/step - loss: 0.4030 - accuracy: 0.8759
Epoch 7/50
25/25 [==============================] - 0s 3ms/step - loss: 0.3515 - accuracy: 0.8924
Epoch 8/50
25/25 [==============================] - 0s 3ms/step - loss: 0.3089 - accuracy: 0.9101
Epoch 9/50
25/25 [==============================] - 0s 3ms/step - loss: 0.2738 - accuracy: 0.9190
Epoch 10/50
25/25 [==============================] - 0s 3ms/step - loss: 0.2449 - accuracy: 0.9241
Epoch 11/50
25/25 [

In [28]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

9/9 - 0s - loss: 0.0181 - accuracy: 0.9962 - 326ms/epoch - 36ms/step
Loss: 0.018120817840099335, Accuracy: 0.9962121248245239


In [29]:
# Export our model to HDF5 file
nn.save('Resources/QChatNN_for_new_survey.h5')

c:\Users\Jessi\anaconda3\envs\dev\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
